In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
# read relevant data
metadata_file = '../DATA/00_stations_metadata.csv'
metadata = pd.read_csv(metadata_file, delimiter=';')

observations_file = '../DATA/00_RS_MN006.csv'
observations = pd.read_csv(observations_file, delimiter=';')

overview_file = '../DATA/00_stations_overview.csv'
overview = pd.read_csv(overview_file, delimiter=';')

In [ ]:
# merge dataframes
merge = pd.merge(metadata, observations, how='inner', on='STATION_ID')
merge = pd.merge(merge, overview, how='inner', on='Stationsname')
merge = merge[['Stationsname', 'east', 'north', 'up', 'ZEITSTEMPEL', 'WERT']]

merge = merge.rename(columns={'Stationsname':'name', 'ZEITSTEMPEL':'timestamp', 'WERT':'value'})
merge = merge.sort_values(['timestamp', 'name'])

In [ ]:
# remove stations outside the timeframe
merge = merge[merge.name != 'Aachen']
merge = merge[merge.name != 'Beckum-Unterberg']
merge = merge[merge.name != 'Bergisch Gladbach -Refrath']
merge = merge[merge.name != 'Dorsten-Hervest']
merge = merge[merge.name != 'Driburg, Bad-Dringenberg']
merge = merge[merge.name != 'Gronau']
merge = merge[merge.name != 'Hamm (Schleuse)']
merge = merge[merge.name != 'Herzogenrath']
merge = merge[merge.name != 'Horn-Meinberg, Bad-Feldrom']
merge = merge[merge.name != 'Kierspe-Höckinghausen']
merge = merge[merge.name != 'Kleve-Schenkenschanz']
merge = merge[merge.name != 'Lippetal-Brockhausen']
merge = merge[merge.name != 'Marienheide-Müllenbach']
merge = merge[merge.name != 'Mettmann']
merge = merge[merge.name != 'Minden-Hahlen']
merge = merge[merge.name != 'Neuss']
merge = merge[merge.name != 'Oberhausen-Klosterhardt']
merge = merge[merge.name != 'Oeynhausen, Bad']
merge = merge[merge.name != 'Schleiden-Morsbach']
merge = merge[merge.name != 'Warendorf']
merge = merge[merge.name != 'Witten-Stockum']
merge = merge[merge.name != 'Übach-Palenberg']
# remove stations with insufficiently measured data
merge = merge[merge.name != 'Rheda-Wiedenbrück-Lintel']
merge = merge[merge.name != 'Nottuln']
merge = merge[merge.name != 'Ennigerloh-Ostenfelde']
merge = merge[merge.name != 'Bocholt-Liedern (Wasserwerk)']
merge = merge[merge.name != 'Porta Westfalica-Barkhausen']
merge = merge[merge.name != 'Aachen-Orsbach']
merge = merge[merge.name != 'Remscheid-Lennep']
merge = merge[merge.name != 'Dorsten-Lembeck']
merge = merge[merge.name != 'Werne-Wessel']
merge = merge[merge.name != 'Stadtlohn']
merge = merge[merge.name != 'Iserlohn-Letmathe']
merge = merge[merge.name != 'Eslohe']
merge = merge[merge.name != 'Fröndenberg-Hohenheide']
merge = merge[merge.name != 'Marsberg-Leitmar']

#### Formatting the timestamp. For clarity only, not relevant for further processing

In [ ]:
# function to reformat date-representation
def reformat_date_representation(df):
    l = len(df)
    cc = 0
    pc = 1
    dates = []
    for idx, row in df.iterrows():
        # compute progress
        cc = cc+1
        per = (cc/l)*100
        if per > pc:
            print('{0}: {1:.1f}%'.format(datetime.now(), pc))
            pc = pc+1
            
        # reformat date-representation
        date = datetime.strptime(str(row['timestamp']), '%Y%m%d').strftime('%Y-%m-%d')
        dates.append(date)
    return pd.Series(dates, name='timestamp')

In [ ]:
# overwrite old timestamp
times = reformat_date_representation(merge)
merge = merge.drop(columns = 'timestamp')
merge = merge.assign(timestamp=times.values)

# resort data by timestamp, name again
merge = merge.sort_values(['timestamp', 'name'])

In [ ]:
# export dataframe as csv
file = '../DATA/01_observations_decoded.csv'
merge.to_csv(file, sep=';', index=False)